In [1]:
import brightway2 as bw2
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

Notebook describes the steps followed to select which of the non-internationally traded commodities are the most relevant to regionalize. 

To do so, we will look at the contributions of each of these non-traded goods in each process of the original ecoinvent database and select the most relevant processes, based on these contributions.

In [2]:
# select the brightway project
bw2.projects.set_current('ecoinvent3.10.1')
db_ecoinvent = bw2.Database('ecoinvent-3.10.1-cutoff')

Which ecoinvent processes to choose for the analysis? All of them, but exclude markets because they are irrelevant. That's about 13,000 processes.

In [3]:
ecoinvent_processes_to_study = []
for act in db_ecoinvent:
    if ('market for' not in act.as_dict()['name'] and 'market group for' not in act.as_dict()['name'] and 
        "import from" not in act.as_dict()['name'] and "generic" not in act.as_dict()['name']):
        ecoinvent_processes_to_study.append(act.key[1])

We need access to the technosphere matrix to do these 13,000 contribution analyses efficiently. The only way to access the technosphere matrix in brightway2 is to make a calculation. So we do a random one.

In [4]:
lca = bw2.LCA({db_ecoinvent.random():1}, ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Human health', 'Total human health'))
lca.lci(factorize=True)
lca.lcia()

Now we will invert that matrix. While this is less efficient than solving the system "à la brightway", for what we are about to do it is a billion times more efficient to just invert the matrix.

In [5]:
A = lca.technosphere_matrix.todense()
I = np.eye(len(A))
A_inv = np.linalg.solve(A, I)

Reformatting A, B and C and its the A inverse. And calculating the lca scores for all the 13,000 processes (that's D)

In [6]:
A = pd.DataFrame(A+I, [i[1] for i in lca.activity_dict], [i[1] for i in lca.activity_dict])
A_inv = pd.DataFrame(A_inv, [i[1] for i in lca.activity_dict], [i[1] for i in lca.activity_dict])
B = pd.DataFrame(lca.biosphere_matrix.todense(), [i[1] for i in lca.biosphere_dict], [i[1] for i in lca.activity_dict])
C = pd.DataFrame(lca.characterization_matrix.todense(), [i[1] for i in lca.biosphere_dict], [i[1] for i in lca.biosphere_dict])
D = C.dot(B).dot(A_inv).sum()

Now if we diagonalize the demand in the technosphere matrix and simply multiply by the normalized life cycle impacts from D, we get the first-tier contributions very efficiently. We only need to add direct emissions as well.

In [7]:
# get first-tier contributions
D_matrix = np.diag(D.values)
Ak = A.loc[:, ecoinvent_processes_to_study]
contribs_hh = D_matrix @ Ak
# change index
contribs_hh.index = lca.activity_dict
# add direct emissions from process
direct_em = C.dot(B).sum()
direct_em = pd.DataFrame(np.diag(direct_em), contribs_hh.index, direct_em.index).T.reindex(contribs_hh.columns).T
contribs_hh -= direct_em

We did it for Total human health. Now switch to ecosystem quality

In [8]:
lca.switch_method(('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Ecosystem quality', 'Total ecosystem quality'))
C = pd.DataFrame(lca.characterization_matrix.todense(), [i[1] for i in lca.biosphere_dict], [i[1] for i in lca.biosphere_dict])
D = C.dot(B).dot(A_inv).sum()

In [9]:
# get first-tier contributions
D_matrix = np.diag(D.values)
Ak = A.loc[:, ecoinvent_processes_to_study]
contribs_eq = D_matrix @ Ak
# change index
contribs_eq.index = lca.activity_dict
# add direct emissions from process
direct_em = C.dot(B).sum()
direct_em = pd.DataFrame(np.diag(direct_em), contribs_eq.index, direct_em.index).T.reindex(contribs_eq.columns).T
contribs_eq -= direct_em

Now let's check the contribution of non-traded products only. So basically remove traded products from the indexes of the contribution matrices

In [10]:
with open('C://Users/11max/PycharmProjects/Regioinvent/Data/Regionalization/ei3.10/ecoinvent_to_HS.json','r') as f:
    regionalized_traded_products = json.load(f)

In [11]:
# need to save totals before cutting through the results
tot_hh = contribs_hh.sum()
tot_eq = contribs_eq.sum()

In [12]:
contribs_hh = contribs_hh.loc[[i for i in contribs_hh.index if db_ecoinvent.get(i[1]).as_dict()['reference product'] not in regionalized_traded_products.keys()]]
contribs_eq = contribs_eq.loc[[i for i in contribs_eq.index if db_ecoinvent.get(i[1]).as_dict()['reference product'] not in regionalized_traded_products.keys()]]

Now regroup per reference product. we don't care if it's sweet gas from sorghum, maize, from the Us or China. We just wanna know if sweet gas is relevant overall.

In [13]:
contribs_hh.index = [db_ecoinvent.get(i[1]).as_dict()['reference product'] for i in contribs_hh.index]
contribs_eq.index = [db_ecoinvent.get(i[1]).as_dict()['reference product'] for i in contribs_eq.index]

In [14]:
contribs_hh = contribs_hh.groupby(level=0).sum()
contribs_eq = contribs_eq.groupby(level=0).sum()

In [15]:
# go to relative results
contribs_hh = contribs_hh / contribs_hh.sum()
contribs_eq = contribs_eq / contribs_eq.sum()

In [16]:
# if the contrib is 1 -> only this product is called in the process -> not real contribution
# same with contribution of 2 with -1 to compensate
contribs_hh = contribs_hh[contribs_hh != 1].fillna(0)
contribs_eq = contribs_eq[contribs_eq != 1].fillna(0)
contribs_hh = contribs_hh[contribs_hh != -1].fillna(0)
contribs_eq = contribs_eq[contribs_eq != -1].fillna(0)
contribs_hh = contribs_hh[contribs_hh != 2].fillna(0)
contribs_eq = contribs_eq[contribs_eq != 2].fillna(0)

In [17]:
# remove the processes that are the basis for the regionalization in regioinvent
regionalization_core = ['heat, district or industrial, natural gas',
                        'heat, district or industrial, other than natural gas',
                        'heat, central or small-scale, other than natural gas',
                        'municipal solid waste',
                        'electricity, for reuse in municipal waste incineration only',
                        'heat, for reuse in municipal waste incineration only',
                        'electricity, high voltage, aluminium industry',
                        'electricity, medium voltage, aluminium industry',
                        'electricity, high voltage, cobalt industry',
                        'electricity, medium voltage, cobalt industry',
                        'electricity, high voltage',
                        'electricity, medium voltage',
                        'electricity, low voltage']
contribs_hh = contribs_hh.loc[~contribs_hh.index.isin(regionalization_core)]
contribs_eq = contribs_eq.loc[~contribs_eq.index.isin(regionalization_core)]


# also remove products that are specific to Swizterland
only_swiss_products = [i for i in contribs_hh.index if 'Swiss' in i] + ["electricity, high voltage, renewable energy products"]
contribs_hh = contribs_hh.loc[~contribs_hh.index.isin(only_swiss_products)]
contribs_eq = contribs_eq.loc[~contribs_eq.index.isin(only_swiss_products)]

In [18]:
# let's order things around
contribs_hh = abs((contribs_hh.sum(axis=1) / contribs_hh.sum().sum())).sort_values(ascending=False)
contribs_eq = abs((contribs_eq.sum(axis=1) / contribs_eq.sum().sum())).sort_values(ascending=False)

If we defined "relevancy" as contributing to more than 1% of contributions overall, then there would be very few non-traded processes that would be relevant: ~15. And I didn't do all of this to regionalize only 15 products, so let's define relevancy at 0.3% instead, this leads to a bit more than 75 products being relevant.

In [19]:
relevant_non_traded_products = list(set(contribs_hh[contribs_hh > 0.003].index.tolist() + contribs_eq[contribs_eq > 0.003].index.tolist()))
relevant_non_traded_products.sort()
relevant_non_traded_products

['blasting',
 'building, hall',
 'building, multi-storey',
 'chemical factory, organics',
 'citric acid',
 'combine harvesting',
 'concrete mixing factory',
 'deep well, drilled, for geothermal power',
 'diesel, burned in building machine',
 'diesel, burned in fishing vessel',
 'drawing of pipe, steel',
 'energy and auxilliary inputs, metal working factory',
 'fertilising, by broadcaster',
 'hazardous waste, for incineration',
 'heat, central or small-scale, natural gas',
 'heat, from steam, in chemical industry',
 'hydropower plant, reservoir, alpine region',
 'hydropower plant, run-of-river',
 'inert material landfill',
 'irrigation',
 'land tenure, arable land, measured as carbon net primary productivity, annual crop',
 'land tenure, arable land, measured as carbon net primary productivity, pasture, man made',
 'land use change, annual crop',
 'land use change, forest, intensive',
 'land use change, pasture, man made',
 'land use change, perennial crop',
 'leachate, SLF, municipal s

In [21]:
with open('C://Users/11max/PycharmProjects/Regioinvent/Data/Regionalization/ei3.10/relevant_non_traded_products.json', 'w') as f:
    json.dump(relevant_non_traded_products, f)